# Compute w'b' 

In [ ]:
import numpy as np
import xarray as xr
from glob import glob
import nc_time_axis
import cftime

In [ ]:
def prepro(ds):
    return ds.isel(y=slice(400, None))

Load grid and data files.  
Note that the $w'$ here has been interpolated to the `deptht` vertical grid before with `c2-interpolate_wprime.ipynb`

In [ ]:
Wprime_data_filesREF = "/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-gridW_prime_interp/*gridWprime.nc"
Wprime_data_filesFUT = "/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-gridW_prime_interp/*gridWprime.nc"

Bprime_data_filesREF = "/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-BUOY_prime/*BUOYprime.nc"
Bprime_data_filesFUT = "/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-BUOY_prime/*BUOYprime.nc"

In [ ]:
grid_files = "/data0/project/drakkar/CONFIGS/CREG12.L75/GRID/CREG12.L75-REF08_mesh_hgr.nc"
coords_file = "/data0/project/drakkar/CONFIGS/CREG12.L75/GRID/coordinates_CREG12_lbclnk_noz_vh20160930.nc"
mask_file= "/data0/project/drakkar/CONFIGS/CREG12.L75/GRID/CREG12.L75-REF08_mask.nc"

In [ ]:
grid = xr.open_mfdataset(grid_files, parallel=True, preprocess=prepro)
coords = xr.open_mfdataset(coords_file, parallel=True, preprocess=prepro)
mask = xr.open_mfdataset(mask_file, parallel=True, preprocess=prepro)

Assign coordinates and chunk data

In [ ]:
Wprime_REF = xr.open_mfdataset(Wprime_data_filesREF, parallel=True, decode_cf=False)
Wprime_REF = Wprime_REF.assign_coords({"nav_lon": grid.nav_lon, "nav_lat": grid.nav_lat})
Wprime_REF = Wprime_REF.chunk({"depthw": 1, "x":395, "y":501})

Wprime_FUT= xr.open_mfdataset(Wprime_data_filesFUT, parallel=True, decode_cf=False)
Wprime_FUT = Wprime_FUT.assign_coords({"nav_lon": grid.nav_lon, "nav_lat": grid.nav_lat})
Wprime_FUT = Wprime_FUT.chunk({"depthw": 1, "x":395, "y":501})

Wprime_REF = Wprime_REF.rename({"depthw":"deptht"})
Wprime_FUT = Wprime_FUT.rename({"depthw":"deptht"})

In [ ]:
Bprime_REF= xr.open_mfdataset(Bprime_data_filesREF, preprocess=prepro, parallel=True,decode_times=False,concat_dim='time_counter', combine='nested')
Bprime_REF = Bprime_REF.assign_coords({"nav_lon":grid.nav_lon,"nav_lat":grid.nav_lat})
Bprime_REF = Bprime_REF.chunk({"deptht": 1, "x":395, "y":501})

Bprime_FUT= xr.open_mfdataset(Bprime_data_filesFUT, preprocess=prepro, parallel=True,decode_times=False,concat_dim='time_counter', combine='nested')
Bprime_FUT = Bprime_FUT.assign_coords({"nav_lon":grid.nav_lon,"nav_lat":grid.nav_lat})
Bprime_FUT = Bprime_FUT.chunk({"deptht": 1, "x":395, "y":501})

Calculate $w'b'$

In [ ]:
wb_p_REF =  Wprime_REF.vovecrtz * Bprime_REF.cv_bn2
wb_p_FUT =  Wprime_FUT.vovecrtz * Bprime_FUT.cv_bn2

Loop over time and save each time step to disk

In [ ]:
output = "/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-W_prime_B_prime/"

for time in wb_p_REF.time_counter:
    date = str(time.values)[0:10]
    print(date)
    year, month, day = date.split("-")
    data = wb_p_REF.sel(time_counter=time)
    data.to_netcdf(output+"CREG12.L75-REF08_y{0}m{1}d{2}.5d_WprimeBprime.nc".format(year,month,day))

In [ ]:
output = "/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-W_prime_B_prime/"

for time in wb_p_FUT.time_counter:
    date = (str(time.values)[0:10])
    print(date)
    year, month, day = date.split("-")
    data = wb_p_FUT.sel(time_counter=time)
    data.to_netcdf(output+"CREG12.L75-FUT08_y{0}m{1}d{2}.5d_WprimeBprime.nc".format(year, month, day))